<a href="https://colab.research.google.com/github/abhishekhbisht11/Heart-Health-Prediction-using-ML/blob/main/HeartHealthPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import warnings
df = pd.read_csv('/content/drive/MyDrive/heart.csv')
df.head()

Age  Sex     ChestPain  RestBP  Chol  Fbs  RestECG  MaxHR  ExAng  Oldpeak  \
0   63    1       typical     145   233    1        2    150      0      2.3   
1   67    1  asymptomatic     160   286    0        2    108      1      1.5   
2   67    1  asymptomatic     120   229    0        2    129      1      2.6   
3   37    1    nonanginal     130   250    0        0    187      0      3.5   
4   41    0    nontypical     130   204    0        2    172      0      1.4   

   Slope  Ca        Thal  Target  
0      3   0       fixed       0  
1      2   3      normal       1  
2      2   2  reversable       1  
3      3   0      normal       0  
4      1   0      normal       0

In [ ]:
print(df.isnull().sum())


Age          0
Sex          0
ChestPain    0
RestBP       0
Chol         0
Fbs          0
RestECG      0
MaxHR        0
ExAng        0
Oldpeak      0
Slope        0
Ca           0
Thal         2
Target       0
dtype: int64


In [ ]:
df=df.dropna()


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Age        303 non-null    int64  
 1   Sex        303 non-null    int64  
 2   ChestPain  303 non-null    object 
 3   RestBP     303 non-null    int64  
 4   Chol       303 non-null    int64  
 5   Fbs        303 non-null    int64  
 6   RestECG    303 non-null    int64  
 7   MaxHR      303 non-null    int64  
 8   ExAng      303 non-null    int64  
 9   Oldpeak    303 non-null    float64
 10  Slope      303 non-null    int64  
 11  Ca         303 non-null    int64  
 12  Thal       301 non-null    object 
 13  Target     303 non-null    int64  
dtypes: float64(1), int64(11), object(2)
memory usage: 33.3+ KB


In [ ]:
df.describe()


Age         Sex      RestBP        Chol         Fbs     RestECG  \
count  303.000000  303.000000  303.000000  303.000000  303.000000  303.000000   
mean    54.438944    0.679868  131.689769  246.693069    0.148515    0.990099   
std      9.038662    0.467299   17.599748   51.776918    0.356198    0.994971   
min     29.000000    0.000000   94.000000  126.000000    0.000000    0.000000   
25%     48.000000    0.000000  120.000000  211.000000    0.000000    0.000000   
50%     56.000000    1.000000  130.000000  241.000000    0.000000    1.000000   
75%     61.000000    1.000000  140.000000  275.000000    0.000000    2.000000   
max     77.000000    1.000000  200.000000  564.000000    1.000000    2.000000   

            MaxHR       ExAng     Oldpeak       Slope          Ca      Target  
count  303.000000  303.000000  303.000000  303.000000  303.000000  303.000000  
mean   149.607261    0.326733    1.039604    1.600660    0.686469    0.458746  
std     22.875003    0.469794    1.161075    0.616226    0.947661    0.499120  
min     71.000000    0.000000    0.000000    1.000000    0.000000    0.000000  
25%    133.500000    0.000000    0.000000    1.000000    0.000000    0.000000  
50%    153.000000    0.000000    0.800000    2.000000    0.000000    0.000000  
75%    166.000000    1.000000    1.600000    2.000000    1.000000    1.000000  
max    202.000000    1.000000    6.200000    3.000000    3.000000    1.000000

In [ ]:
d = df['Target'].value_counts()
print(d)

0    164
1    139
Name: Target, dtype: int64


In [ ]:
import seaborn as sns
def plotTarget():
    sns.countplot(x='Target', data=df, ax=ax)
    for i, p in enumerate(ax.patches):
        count=df['Target'].value_counts().values[i]
        x=p.get_x()+ p.get_width() /2.
        y=p.get_height() + 3
        label='{:1.2f}'.format(count / float(df.shape[0]))
        ax.text(x, y,label, ha='center')

fig_target,ax=plt.subplots(nrows=1, ncols=1, figsize=(5, 2))
plotTarget()


In [ ]:
df.corr()


<ipython-input-10-2f6f6606aa2c>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


Age       Sex    RestBP      Chol       Fbs   RestECG     MaxHR  \
Age      1.000000 -0.098138  0.284734  0.208287  0.121670  0.149037 -0.395982   
Sex     -0.098138  1.000000 -0.065284 -0.202126  0.041025  0.029016 -0.057065   
RestBP   0.284734 -0.065284  1.000000  0.129371  0.178498  0.147089 -0.046402   
Chol     0.208287 -0.202126  0.129371  1.000000  0.015762  0.171185 -0.005690   
Fbs      0.121670  0.041025  0.178498  0.015762  1.000000  0.079892 -0.012297   
RestECG  0.149037  0.029016  0.147089  0.171185  0.079892  1.000000 -0.077950   
MaxHR   -0.395982 -0.057065 -0.046402 -0.005690 -0.012297 -0.077950  1.000000   
ExAng    0.092985  0.140802  0.065564  0.064250  0.013534  0.092626 -0.386043   
Oldpeak  0.203604  0.098482  0.188801  0.044836  0.004855  0.117580 -0.349391   
Slope    0.162228  0.031571  0.117437 -0.004228  0.054079  0.140144 -0.393527   
Ca       0.331939  0.100345  0.100535  0.106125  0.164689  0.127330 -0.256365   
Target   0.224394  0.272006  0.151471  0.086762  0.015613  0.177049 -0.425870   

            ExAng   Oldpeak     Slope        Ca    Target  
Age      0.092985  0.203604  0.162228  0.331939  0.224394  
Sex      0.140802  0.098482  0.031571  0.100345  0.272006  
RestBP   0.065564  0.188801  0.117437  0.100535  0.151471  
Chol     0.064250  0.044836 -0.004228  0.106125  0.086762  
Fbs      0.013534  0.004855  0.054079  0.164689  0.015613  
RestECG  0.092626  0.117580  0.140144  0.127330  0.177049  
MaxHR   -0.386043 -0.349391 -0.393527 -0.256365 -0.425870  
ExAng    1.000000  0.287926  0.254076  0.151731  0.427860  
Oldpeak  0.287926  1.000000  0.576795  0.274451  0.423894  
Slope    0.254076  0.576795  1.000000  0.102041  0.335806  
Ca       0.151731  0.274451  0.102041  1.000000  0.454822  
Target   0.427860  0.423894  0.335806  0.454822  1.000000

In [ ]:
def plotAge():
    facet_grid = sns.FacetGrid(df, hue='Target')
    facet_grid.map(sns.kdeplot, "Age", shade=True, ax=axes[0])
    legend_labels = ['Disease false', 'Disease true']
    for t, l in zip(axes[0].get_legend().texts, legend_labels):
        t.set_text(l)
        axes[0].set(xlabel='Age', ylabel='Density')

    avg = df[["Age", "Target"]].groupby(['Age'], as_index=False).mean()
    sns.barplot(x='Age', y='Target', data=avg, ax=axes[1])
    axes[1].set(xlabel='Age', ylabel='disease probability')

    plt.clf()

fig_age, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 8))
plotAge()


NameError: name 'sns' is not defined

In [ ]:
x=df['ChestPain']
x.value_counts()


asymptomatic    143
nonanginal       85
nontypical       50
typical          23
Name: ChestPain, dtype: int64

In [ ]:
x=df['Thal']
x.value_counts()


normal        166
reversable    117
fixed          18
Name: Thal, dtype: int64

In [ ]:
import seaborn as sns
category=[('ChestPain', ['typical','nontypical','nonanginal','asymptomatic']),('Thal',['fixed','normal','reversable',])]
continuous = [('Age', 'Age in year'),
              ('Sex','1 for Male 0 for Female'),
              ('RestBP','BP in Rest State'),
              ('Fbs','Fasting blood glucose'),
              ('RestECG','ECG at rest'),
              ('Chol', 'serum cholestoral in mg/d'),
              ('MaxHR','Max Heart Rate'),
              ('ExAng','Exchange Rate'),
              ('Slope','Slope of Curve'),
              ('Oldpeak', 'ST depression by exercise relative to rest'),
              ('Ca', '# major vessels: (0-3) colored by flourosopy')]


def plotCategorial(attribute, labels, ax_index):
    sns.countplot(x=attribute, data=df, ax=axes[ax_index][0])
    sns.countplot(x='Target', hue=attribute, data=df, ax=axes[ax_index][1])
    avg = df[[attribute, 'Target']].groupby([attribute], as_index=False).mean()
    sns.barplot(x=attribute, y='Target', hue=attribute, data=avg, ax=axes[ax_index][2])

    for t, l in zip(axes[ax_index][1].get_legend().texts, labels):
        t.set_text(l)
    for t, l in zip(axes[ax_index][2].get_legend().texts, labels):
        t.set_text(l)


def plotContinuous(attribute, xlabel, ax_index):
    sns.distplot(df[[attribute]], ax=axes[ax_index][0])
    axes[ax_index][0].set(xlabel=xlabel, ylabel='density')
    sns.violinplot(x='Target', y=attribute, data=df, ax=axes[ax_index][1])


def plotGrid(isCategorial):
    if isCategorial:
        [plotCategorial(x[0], x[1], i) for i, x in enumerate(category)]
    else:
        [plotContinuous(x[0], x[1], i) for i, x in enumerate(continuous)]


In [ ]:
fig_categorial,axes=plt.subplots(nrows=len(category), ncols=3, figsize=(10, 10))
plotGrid(isCategorial=True)


In [ ]:
fig_continuous, axes = plt.subplots(nrows=len(continuous), ncols=2, figsize=(10,10))
plotGrid(isCategorial=False)


<ipython-input-14-4a0b182287d7>:29: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(df[[attribute]], ax=axes[ax_index][0])
<ipython-input-14-4a0b182287d7>:29: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(df[[attribute]], ax=axes[ax_index][0])
<ipython-input-14-4a0b182287d7>:29: UserWarning: 

`di

In [ ]:
#dummy for chest Pain
chestpain_dummy = pd.get_dummies(df.loc[:,'ChestPain'])
chestpain_dummy.rename(columns={1: 'Typical', 2: 'Asymptomatic',3: 'Nonanginal', 4: 'Nontypical'}, inplace=True)
#dummy for RestECG
restecg_dummy = pd.get_dummies(df.loc[:,'RestECG'])
restecg_dummy.rename(columns={0: 'Normal_restECG', 1: 'Wave_abnormal_restECG',2:'Ventricular_ht_restECG'},inplace=True)
#dummy for Slope
slope_dummy = pd.get_dummies(df['Slope'])
slope_dummy.rename(columns={1: 'Slope_upsloping', 2:'Slope_flat',3: 'Slope_downsloping'},inplace=True)
#dummy for Thal
thal_dummy = pd.get_dummies(df['Thal'])
thal_dummy.rename(columns={3: 'Thal_Normal', 6: 'Thal_fixed',7: 'Thal_reversible'}, inplace=True)
#concatination in data frame
df = pd.concat([df,chestpain_dummy, restecg_dummy, slope_dummy, thal_dummy], axis=1)
#droping Column because their dummies are created
df.drop(['ChestPain','RestECG', 'Slope', 'Thal'], axis=1, inplace=True)


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 301 entries, 0 to 302
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     301 non-null    int64  
 1   Sex                     301 non-null    int64  
 2   RestBP                  301 non-null    int64  
 3   Chol                    301 non-null    int64  
 4   Fbs                     301 non-null    int64  
 5   MaxHR                   301 non-null    int64  
 6   ExAng                   301 non-null    int64  
 7   Oldpeak                 301 non-null    float64
 8   Ca                      301 non-null    int64  
 9   Target                  301 non-null    int64  
 10  asymptomatic            301 non-null    uint8  
 11  nonanginal              301 non-null    uint8  
 12  nontypical              301 non-null    uint8  
 13  typical                 301 non-null    uint8  
 14  Normal_restECG          301 non-null    ui

In [ ]:
df.head()


Age  Sex  RestBP  Chol  Fbs  MaxHR  ExAng  Oldpeak  Ca  Target  ...  \
0   63    1     145   233    1    150      0      2.3   0       0  ...   
1   67    1     160   286    0    108      1      1.5   3       1  ...   
2   67    1     120   229    0    129      1      2.6   2       1  ...   
3   37    1     130   250    0    187      0      3.5   0       0  ...   
4   41    0     130   204    0    172      0      1.4   0       0  ...   

   typical  Normal_restECG  Wave_abnormal_restECG  Ventricular_ht_restECG  \
0        1               0                      0                       1   
1        0               0                      0                       1   
2        0               0                      0                       1   
3        0               1                      0                       0   
4        0               0                      0                       1   

   Slope_upsloping  Slope_flat  Slope_downsloping  fixed  normal  reversable  
0                0           0                  1      1       0           0  
1                0           1                  0      0       1           0  
2                0           1                  0      0       0           1  
3                0           0                  1      0       1           0  
4                1           0                  0      0       1           0  

[5 rows x 23 columns]

In [ ]:
df_X= df.loc[:, df.columns != 'Target']
df_y= df.loc[:, df.columns == 'Target']



In [ ]:
import statsmodels.api as sm
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
selected_features=[]
lr=LogisticRegression()
rfe=RFE(lr)

warnings.simplefilter('ignore')
rfe.fit(df_X.values,df_y.values)
print(rfe.support_)
print(rfe.ranking_)

for i, feature in enumerate(df_X.columns.values):
    if rfe.support_[i]:
        selected_features.append(feature)

df_selected_X = df_X[selected_features]
df_selected_y=df_y

lm=sm.Logit(df_selected_y,df_selected_X)
result = lm.fit()

print(result.summary2())
warnings.simplefilter('ignore')


[False  True False False False False  True  True  True  True  True False
  True  True False False False  True False False  True  True]
[10  1 11 12  3  9  1  1  1  1  1  7  1  1  5  6  2  1  8  4  1  1]
Optimization terminated successfully.
         Current function value: 0.352807
         Iterations 7
                         Results: Logit
Model:              Logit            Method:           MLE       
Dependent Variable: Target           Pseudo R-squared: 0.488     
Date:               2023-12-08 06:09 AIC:              234.3899  
No. Observations:   301              BIC:              275.1681  
Df Model:           10               Log-Likelihood:   -106.19   
Df Residuals:       290              LL-Null:          -207.60   
Converged:          1.0000           LLR p-value:      4.1933e-38
No. Iterations:     7.0000           Scale:            1.0000    
-----------------------------------------------------------------
                   Coef.  Std.Err.    z    P>|z|   [0.025  0.

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test=train_test_split(df_selected_X,df_selected_y, test_size = 0.25, random_state =0)
columns = X_train.columns



In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
def cal_accuracy(y_test, y_predict):

    print("\nConfusion Matrix: \n",
    confusion_matrix(y_test, y_predict))

    print (f"\nAccuracy : {accuracy_score(y_test,y_predict)*100:0.3f}")


In [ ]:
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_predict=lr.predict(X_test)
print(f"Accuracy of Test Dataset: {lr.score(X_test,y_test):0.3f}")
print(f"Accuracy of Train Dataset: {lr.score(X_train,y_train):0.3f}")
warnings.simplefilter('ignore')


Accuracy of Test Dataset: 0.882
Accuracy of Train Dataset: 0.840


In [ ]:
print("Predicted values:")
print(y_predict)
cal_accuracy(y_test, y_predict)


Predicted values:
[1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 1 1 1 1 0 0 0 1 0 0 1
 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 1 0 1 0 0 0 1 1 0 1 1 1 1 0 0 0 1
 1 0]

Confusion Matrix: 
 [[37  3]
 [ 6 30]]

Accuracy : 88.158


In [ ]:
from sklearn import svm
svm_linear = svm.SVC(kernel='linear')
svm_linear.fit(X_train,y_train)
warnings.simplefilter('ignore')
print(f"Accuracy of Test Dataset: {svm_linear.score(X_test,y_test):0.3f}")
print(f"Accuracy of Train Dataset: {svm_linear.score(X_train,y_train):0.3f}")


Accuracy of Test Dataset: 0.882
Accuracy of Train Dataset: 0.853


In [ ]:
print("Predicted values:")
print(y_predict)
cal_accuracy(y_test, y_predict)


Predicted values:
[1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 1 1 1 1 0 0 0 1 0 0 1
 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 1 0 1 0 0 0 1 1 0 1 1 1 1 0 0 0 1
 1 0]

Confusion Matrix: 
 [[37  3]
 [ 6 30]]

Accuracy : 88.158


In [ ]:
from sklearn.tree import DecisionTreeClassifier
gini = DecisionTreeClassifier(criterion = "gini", random_state =100,max_depth=3, min_samples_leaf=5)
gini.fit(X_train, y_train)
warnings.simplefilter('ignore')
print(f"Accuracy of Test Dataset: {gini.score(X_test,y_test):0.3f}")
print(f"Accuracy of Train Dataset: {gini.score(X_train,y_train):0.3f}")


Accuracy of Test Dataset: 0.855
Accuracy of Train Dataset: 0.849


In [ ]:
y_predict=gini.predict(X_test)
print("Predicted values:\n")
print(y_predict)
cal_accuracy(y_test, y_predict)


Predicted values:

[1 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 1 1 0 1 1 1 1 1 0 0 1 0 0 1
 0 0 1 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1 1 1 0 1 0 1 0 1 1 0 1 1 1 1 0 0 1 0
 0 0]

Confusion Matrix: 
 [[36  4]
 [ 7 29]]

Accuracy : 85.526


In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest=RandomForestClassifier(n_estimators=100)
forest.fit(X_train,y_train)

warnings.simplefilter('ignore')
print(f"Accuracy of Test Dataset: {forest.score(X_test,y_test):0.3f}")
print(f"Accuracy of Train Dataset: {forest.score(X_train,y_train):0.3f}")


Accuracy of Test Dataset: 0.855
Accuracy of Train Dataset: 0.973


In [ ]:
y_predict=forest.predict(X_test)
print("Predicted values:\n")
print(y_predict)
cal_accuracy(y_test, y_predict)


Predicted values:

[1 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 1 1 1 1 1 1 1 0 0 0 1 0 0 1
 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 1 0 1 0 1 0 1 1 0 0 1 1 1 0 0 1 0
 1 0]

Confusion Matrix: 
 [[35  5]
 [ 6 30]]

Accuracy : 85.526


In [ ]:
models=pd.DataFrame({'Model':['Logistics Regression','SVM','Decision Tree','Random Forest'],
                     'Traning Accuracy':[(lr.score(X_train,y_train)),svm_linear.score(X_train,y_train),gini.score(X_train,y_train),forest.score(X_train,y_train)],
                     'Test Accuracy':[(lr.score(X_test,y_test)),svm_linear.score(X_test,y_test),gini.score(X_test,y_test),forest.score(X_test,y_test)]})
models.sort_values(by='Test Accuracy', ascending=False)


Model  Traning Accuracy  Test Accuracy
0  Logistics Regression          0.840000       0.881579
1                   SVM          0.853333       0.881579
2         Decision Tree          0.848889       0.855263
3         Random Forest          0.973333       0.855263

In [ ]:
print('Best Model is Logistic Regression')

Best Model is Logistic Regression
